# AREOMMA NASA DC-8 Aircraft Data to CMAQ Pairing

---
    author: Michael J. Pye (pye.michael@epa.gov)
    date created: 2025-01-10
---

### Origin
This notebook is based upon another notebook written by Havala Pye (pye.havala@epa.gov) for the FIREX field campaign created on 2023-01-10.  
The path to the original notebook is: /work/MOD3DEV/has/2022firex/postproc/notebooks/2024_firexpairv3_hotp_20240304_base3.ipynb  
Although this notebook has a very different structure, at least some of the ideas for this notebook came from that notebook. 

### Purpose
This notebook is designed to perform flight pairing tasks specific to the AEROMMA aircraft observations. 

### Log
* 01/10/2025 - added an import of scipy to the check install version cell
* 01/10/2025 - added section to read in merge.csv files as opposed to icartt files
* 01/13/2025 - started the file fresh. Revoved all code and rearranged the introduction section
* 01/13/2025 - added `find_one_minute_coords()`, a function that finds x, y, p, and t from any AEROMMA flight and aggregates the data from a one second time step to a one minute time step.
* 01/13/2025 - added `flight_id_creator()`, a function that creates a flight ID based on the name of the AEROMMA merge file
* 01/16/2025 - updated the `find_one_minute_coords()` function to remove all one second observations missing at least one coordinate value
* 01/17/2025 - split the `find_one_minute_coords()` function into two sepearate functions; one to find coordinates based on the input merge file to address the fact that some files have different coordinate data sets, and another to aggregate the one second data into one minute time steps.
* 01/21/2025 - addressed issues in the function for aggregating one second data to one minute time steps.
* 01/30/2025 - added a function to compare coordinate data in CMAQ and from AEROMMA to see how well they line up.
* 01/30/2025 - added a function to plot paired AEROMMA and CMAQ data of the same variable to look for outliers in the matches.
* 02/03/2025 - Added a function to extract flight data in addition to just the coordinate data
* 02/04/2025 - Added a function that prints a list of variables in an AEROMMA merge file based on the path of the merge file
* 02/07/2025 - Added a function that removes time steps from both AEROMMA and CMAQ DataFrames for times when the aircraft is outside the model domain
* 02/07/2025 - Replaced the `one_minute_data()` function with the `time_agg_data()` function. The function works the same way except the `time_agg_data()` allows for multiple time interval options as opposed to just one minute.
* 02/07/2025 - added a function to remove negative values from columns that should not have them.
* 03/11/2025 - Began writing a function to consolidate a fully cleaned flight DataFrame (CMAQ or AEROMMA) into a DataFrame where each row represents an altitude bin.
* 03/24/2025 - Completed a function that plots the variation of data in the vertical
* 03/28/2025 - Added a function to remove CMAQ data for times when AEROMMA data is missing. This only operates on a single column of data as opposed to an entire DataFrame.
* 04/02/2025 - Added a function to plot the spatial variation of differences between CMAQ and AEROMMA data.
* 04/04/2025 - Added a function to plot time series of CMAQ and/or AEROMMA data.
* 04/07/2025 - Added a funtion that produces a table of statistics to compare CMAQ and AEROMMA data
* 04/10/2025 - Added a function to create a scatter plot of any variable as a function of hydrogen cyanide concentration. Hydrogen cyanide is common in wildfire smoke, which means plotting other variables as a function of HCN could help isolate if CMAQ is performing well or not in wildfire smoke plumes.
---

# Imports

In [ ]:
#Import Packages
import numpy as np
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plot
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib import gridspec
from warnings import warn
import scipy.stats as sp_stat

#Import Jupyter Notebooks
import import_ipynb
import cmaq_flight_pairing

# Prepare AEROMMA data for pairing
### Find flight coordinates
The `find_coords()` function opens an AEROMMA Merge file and exctracts the coordinate data. The coordinates extracted are:  
* GPS Longitude [Degrees] (Converted from degrees * 10^5.)
* GPS Latitude [Degrees] (Converted from degrees * 10^5)
* GPS Altitude Above Sea Level [meters] (Converted from decimeters)
* Time [Format: YYYY-MM-DD HH:MM:SS]  

Parameters:
* `mrg_file_path` (str) - The path of the AEROMMA merge file for the desired flight. Can be a relative or absolute path, but absolute is recommended. 

Returns:  
* `coord_df` (pandas.Dataframe) - DataFrame that stores the coordinate data under keys: `'G_LONG'`, `'G_LAT'`, and `'G_ALT'`. Each row in the dataframe is indexed by the time stamp of the AEROMMA observation. 

In [ ]:
def find_coords(mrg_file_path):
    #open merge file and #extract coordinate data
    mrg_data = pd.read_csv(mrg_file_path)
    sec_times = mrg_data['Time'].values   #times have a format of YYYY-MM-DD HH:MM:SS and are in string format
    sec_lons = mrg_data['G_LONG'].values / (10 ** 5)   #values are positive in the eastern hemisphere and negative in the western hemisphere. values are stored after having been multiplied by 10^5
    sec_lats = mrg_data['G_LAT'].values / (10 ** 5)   #values are positive in the northern hemisphere and negative in the southern hemisphere. values are stored after having been multiplied by 10^5
    sec_alts = mrg_data['G_ALT'].values * 0.1    #values are originally in units of decimeters and represent height above sea level. They are then converted to meters by multiplying by 0.1 m/dm   
    coord_df = pd.DataFrame({'G_LONG':sec_lons, 'G_LAT':sec_lats, 'G_ALT':sec_alts, 'Time':sec_times})
    return coord_df.set_index('Time')

### Aggregate to user defined timestep
The purpose of the `time_agg_data()` function is to take a pandas DataFrame with data at a one second time step, that includes the keys `'Time'`, `'G_LONG'`, `'G_LAT'`, and `'G_ALT'`, and converts it to have a time step of a desired length.  

Parameters:
* `flight_df` (pandas.DataFrame) - DataFrame containing at least flight coordinate data and with a time step of less than a minute. The index must be the time stamp of each observation. 
* `agg_timestep` (int) [Default: `60`] - number of minutes or seconds for each aggregated time interval. This must be a factor of 60. Possible options are: 1, 2, 3, 4, 5, 6, 10, 12, 15, 30, and 60. Any other value will raise a `ValueError`.
* `time_units` (str) [Default: `'sec'`] - units of the aggregated time step. possible options are `'sec'` (for seconds) and `'min'` (for minutes)
  
Returns:
* `agg_df` (pandas.DataFrame) - DataFrame that contains the same data as the input DataFrame but aggregated to a user defined time step and with the index based on the `'Time'` merge variable. The coordinate data is always present for all rows of the output DataFrame. When a variable has no data for a given timestep, the value for that time is marked with numpy.nan

In [ ]:
def time_agg_data(flight_df, agg_timestep = 60, time_units = 'sec'):
    if agg_timestep in [1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 30, 60]:    #make sure agg_timestep is always a factor of 60
        col_keys = flight_df.keys()
        ob_ct = {}    #keeps track of the number of observations for each individual variable during one aggregation timestep
        agg_data = {}    #stores arrays of aggregated data for all variables
        agg_data['Time'] = []
        for key in col_keys:
            agg_data[key] = [0]
            ob_ct[key] = 0

        #iterate through all observations
        for i in range(len(flight_df['G_LAT'].values)):
            missing_coord_check = [pd.isnull(coord) for coord in [flight_df['G_LONG'].values[i], flight_df['G_LAT'].values[i], flight_df['G_ALT'].values[i]]]
            if True not in missing_coord_check:
                current_round_time = round_time(flight_df.index[i], agg_timestep, time_units)    #see the explination for the round_time() function below
                if 'last_round_time' not in locals():
                    last_round_time = round_time(flight_df.index[i], agg_timestep, time_units)    #see the explination for the round_time() function below

                #if the round time has not changed since the previous observation, add current values to the timestep total
                if current_round_time == last_round_time:
                    for key in col_keys:
                        try:
                            current_value = np.array(flight_df[key])[i]
                            int(current_value)    #if the current value is a nan value, this line will cause a ValueError to be raised. This will prevent the missing data from being added to the agg_data array and will stop an additional count from occuring in the ob_ct dict for the variable
                            agg_data[key][-1] += current_value
                            ob_ct[key] += 1
                        except ValueError:
                            pass

                #if the round_time has changed since the previous observation, divide the sum of the values by the number
                #of observations taken that timestep, then add the current values to a new runing total
                elif current_round_time != last_round_time:
                    agg_data['Time'].append(last_round_time)
                    for key in col_keys:
                        if ob_ct[key] > 0:    #if values exist for the timestep, take the average of those values
                            agg_data[key][-1] /= ob_ct[key]
                        else:    #if values do not exist for the timestep, set the value of the for the timestep to np.nan
                            agg_data[key][-1] = np.nan
                    for key in col_keys:    #prep lists for new data and add first data point of the timestep
                        agg_data[key].append(0)
                        ob_ct[key] = 0
                        try:
                            current_value = np.array(flight_df[key])[i]
                            int(current_value)
                            agg_data[key][-1] += current_value
                            ob_ct[key] += 1
                        except ValueError:
                            pass

                last_round_time = current_round_time 

        #determine if there is any data left over that needs to be included and averaged
        present_data = False
        for key in col_keys:
            if ob_ct[key] != 0:
                present_data = True
        
        #take the average of remaining data is there is any.
        if present_data == True:
            agg_data['Time'].append(last_round_time)
            for key in col_keys:
                if ob_ct[key] > 0:    #if values exist for the timestep, take the average of those values
                    agg_data[key][-1] /= ob_ct[key]
                else:    #if values do not exist for the timestep, set the value of the for the timestep to np.nan
                    agg_data[key][-1] = np.nan

        agg_df = pd.DataFrame(agg_data).set_index('Time')
        return agg_df
    
    #if agg_timestep is not a factor of 60, raise a ValueError to prevent further issues
    else:
        raise ValueError('AEROMMA Flight Pairing: The value entered for "agg_timestep" is not a factor of 60.')


###Extra function for use in time_agg_data()
#this function creates a string that rounds the time down to the closest time data is getting aggregated to.
#For example, if the time_str contains 19:25:39, the agg_timestep is 20, and the time units is "sec", the 
#out_time would contain 19:25:20. This would aggregate the data from 19:25:39 to the 19:25:20 timestep. If 
#the time was changed to 19:45:43, out_time would be 19:45:40. If the time_units parameter was changed to 
#"min", the out_time values for the same times would be 19:20:00 and 19:40:00.
def round_time(time_str, agg_timestep, time_units):
    if time_units == 'min':
        min_value = int(time_str[-5:-3])    #number of minutes past the hour
        min_factor = int(min_value / agg_timestep)
        out_time = datetime.datetime.strptime(time_str[:-6], '%Y-%m-%d %H') + (datetime.timedelta(minutes = agg_timestep) * min_factor)
    elif time_units == 'sec':
        sec_value = int(time_str[-2:])     #number of seconds past the minute 
        sec_factor = int(sec_value / agg_timestep)
        out_time = datetime.datetime.strptime(time_str[:-3], '%Y-%m-%d %H:%M') + (datetime.timedelta(seconds = agg_timestep) * sec_factor)
    return out_time.strftime('%Y-%m-%d %H:%M:%S')

### Create a unique ID for the flight
The purpose of the `flight_id_creator()` function is to create a string ID that is unique to the flight so it can be added as the name of a netcdf variable. It can also be used to find what a flight ID within an already generated netCDF should be based on the flight name. It takes the path of the merge file and strips the flight_specific information from the file name and creates an ID based on that.  
  
Parameters:  
* `mrg_file_path` (str) - The path of the AEROMMA merge file for the desired flight. Can be a relative or absolute path, but absolute is recommended.  
  
Returns:
* `flight_id` (str) - The flight ID. This can be used to give to a NetCDF file to create a variable name, or to replicate what the NetCDF likely has stored as the variable for the flight at hand.


In [ ]:
def flight_id_creator(mrg_file_path):
    #Get Merge file name and remove all parts of the file name not specific to the flight
    mrg_file_name = os.path.basename(mrg_file_path)
    split = mrg_file_name.split('_')
    split.remove('AEROMMA')
    split.remove('Merge.csv')
    
    #Create the flight ID based on the remaining parts of the Merge file name
    if len(split) == 1:
        flight_id = 'flight_' + split[0]    #This is specific to when there is exactly one flight in a day
    elif len(split) == 2:
        flight_id = 'flight_' + split[0] + '_' + split[1]    #This is specific to when there are exactly two flights in a day
    return flight_id

### Pair AEROMMA data to CMAQ
The purpose of the `pair_aeromma_flight()` function is to add the CMAQ indices associated with an AEROMMA flight trajectory to a netCDF file based on the path of the AEROMMA merge file, the path of the netCDF file, and the directory storing the CMAQ data.  

Parameters:
* `mrg_file_path` (str) - path to the AEROMMA merge file
* `index_file_path` (str or NoneType) - path to the netCDF file where the indices are to be stored. If the file does not exist yet, enter `None`.
* `domain_dir` (str) - The absolute path of the directory where the GRIDCRO2D and METCRO3D are stored. 
* `model_resolution` (int) [Default: `12000`] - The horizontal grid spacing of the model in units of meters. This is only used to tell the function if the aircraft has left the model domain. Set this value to limit the farthest distance from a model grid point that you would like flight data to pair to model data. 
* `alt_type` (str) [Default: `'ASL'`] - specifies whether the altitudes passed to the function are in height above ground level or height above sea level. the options are `'ASL'` (above sea level) and `'AGL'` (above ground level) 

In [ ]:
def pair_aeromma_flight(mrg_file_path, index_file_path, domain_dir, model_resolution = 12000, alt_type = 'AGL'):
    sec_coords = find_coords(mrg_file_path)
    min_coords = time_agg_data(sec_coords)
    flight_id = flight_id_creator(mrg_file_path)
    cmaq_flight_pairing.pair_flight(flight_id, min_coords['G_LONG'], min_coords['G_LAT'], min_coords['G_ALT'], min_coords.index, index_file_path, domain_dir, model_resolution = model_resolution, alt_type = alt_type)

The purpose of the `pair_all_aeromma_flights()` function is to perform the actions of the `pair_aeromma_flight()` function for all AEROMMA merge files within a directory.  

Parameters:  
* `mrg_dir_path` (str) - Path of the directory containing multiple AEROMMA Merge files or multiple directries that contain a Merge file. If the Merge files are in sub-directories of the mrg_dir_path, the Merge file name must be the same as the sub-directory name but with a `.csv` at the end. For example: `mrg_dir_path/AEROMMA_20230614_Merge/AEROMMA_20230614_Merge.csv`.
* `index_file_path` (str) - Path to the netCDF file where the indices are to be stored.
* `sub_directory` (bool) [Default: `True`] - Tells the function whether the Merge files are directly in the `mrg_dir_path` directory or in sub-directories of the `mrg_dir_path` directory.

In [ ]:
def pair_all_aeromma_flights(mrg_dir_path, index_file_path, sub_directory = True):
    for flight_dir in sorted(os.listdir(mrg_dir_path)):
        if 'AEROMMA' in flight_dir:
            if sub_directory == True:
                mrg_file_path = mrg_dir_path + '/' + flight_dir + '/' + flight_dir + '.csv'
            elif sub_directory == False:
                mrg_file_path = mrg_dir_path + '/' + flight_dir + '.csv'
            if os.path.exists(index_file_path) == True:
                pair_aeromma_flight(mrg_file_path, index_file_path)
            elif os.path.exists(index_file_path) == False:
                pair_aeromma_flight(mrg_file_path, None)

# Extract AEROMMA Data

### List AEROMMA merge file variable names
Shows the user what variables are available in an AEROMMA merge file.  
  
Parameters:  
* `file_path` (str) - path of the AEROMMA merge file which the variable names should be read from
* `rows` (bool) - tells the function whether to print each variable name on a new row (enter `True` for this option) or to just print out the whole list of variable names in one go (enter `False` for this option). 

In [ ]:
def list_vars(file_path, rows = False):
    flight_data = pd.read_csv(file_path)
    var_list = list(flight_data.keys())
    
    if rows == True:
        for var_name in var_list:
            print(var_name)
    elif rows == False:
        print(var_list)

### Extract desired AEROMMA variables
The purpose of the `extract_flight_data()` function is to pull out all wanted variables from an AEROMMA merge file and put them into a pandas DataFrame indexed by time.  
  
Parameters:
* `mrg_file_path` (str) - The path of the AEROMMA merge file for the desired flight. Can be a relative or absolute path, but absolute is recommended.
* `output_vars` (list or NoneType) [Default: `None`] - list of AEROMMA variables to be extracted. If left as None, all AEROMMA variables will be extracted.  

Returns:  
* `sec_data` (pandas.DataFrame) - Contains flight coordinate data in addition to all desired AEROMMA variables indexed by observation timestamp.

In [ ]:
def extract_flight_data(mrg_file_path, output_vars = None):
    if output_vars == None:    #If no variables are listed for specific extraction, all variables are extracted here
        sec_data = pd.read_csv(mrg_file_path).set_index('Time')
    elif output_vars != None:    #If extraction variables are listed, this section extracts these specific variables in addition to coordinate variables
        sec_data = find_coords(mrg_file_path)
        extra = pd.read_csv(mrg_file_path).set_index('Time')
        for var_name in output_vars:
            try:
                sec_data[var_name] = extra[var_name][:]
            except KeyError:
                sec_data[var_name] = np.nan
                warn(var_name + ' not found in "' + mrg_file_path + '". Setting all values in column to np.nan.')
    return sec_data

# QC AEROMMA Data

### Remove negative values that do not belong in the 1 Hz data
The purpose of the `rm_1hz_negatives()` function is to remove negative values from data columns that should not have negative values. For the columns that should not have negative values, this function replaces all negative values with `numpy.nan`. There are some variables in AEROMMA Merge files that should have negative values included. These values are included in the `ignore_cols` list. However, this is not an exhaustive list of all variables that can have negatives. Additional variables can be ignored by the negative removal process by adding the variable names to the `extra_ignore` list.  

Parameters:  
* `sec_data` (pandas.DataFrame) - Contains flight coordinate data in addition to all desired AEROMMA variables indexed by observation timestamp.
* `extra_ignore` (list) [Default: `[]`] - List of AEROMMA variables that should not under go the process removing negative values in addition to the standard list of variables: `['G_LAT', 'G_LONG', 'ROLL', 'PITCH', 'YAW', 'AOA', 'U', 'V', 'W']`.  

Returns:
* `sec_data` (pandas.DataFrame) - Contains flight coordinate data in addition to all desired AEROMMA variables indexed by observation timestamp and has negative values removed from columns that should not have them.

In [ ]:
def rm_1hz_negatives(sec_data, extra_ignore = []):
    #create a list of variables that normally have negative values. Allow the user to add extra variables 
    #that are not included in the basic "ignore_cols" list. 
    ignore_cols = ['G_LAT', 'G_LONG', 'ROLL', 'PITCH', 'YAW', 'AOA', 'U', 'V', 'W']
    for var_name in extra_ignore:
        ignore_cols.append(var_name)
    
    #replace negative values with np.nan in columns that do not normally have negatives
    for key in sec_data.keys():
        if key not in ignore_cols:
            sec_data[key] = sec_data[key].apply(lambda x: np.nan if x < 0 else x)
    
    return sec_data

### Remove times where aircraft is outside model domain
the purpose of the `rm_outside_model_times()` function is to create a list of times where the CMAQ extraction output has marked an AEROMMA observtion as taking place outside the model domain. Using that list of times, it removes the associated values from all variables in both the CMAQ and AEROMMA DataFrames.  

Parameters:  
* `cmaq_df` (pandas.DataFrame) - DataFrame containing CMAQ data paired to an AEROMMA flight
* `agg_df` (pandas.DataFrame) - DataFrame of AEROMMA flight data aggregated to a regular timestep  

Returns:
* `cmaq_df` (pandas.DataFrame) - DataFrame containing CMAQ data paired to an AEROMMA flight. Data from times where the aircraft was outside the model domain has been removed.
* `agg_df` (pandas.DataFrame) - DataFrame of AEROMMA flight data aggregated to a regular timestep. Data from times where the aircraft was outside the model domain has been removed.

In [ ]:
def rm_outside_model_times(cmaq_df, agg_df):
    #If the aircraft is outside the CMAQ domain, all values in the row are marked with -9999.9999
    #bad_times is essentially a list of times when all data frame row values are -9999.9999
    bad_times = list(cmaq_df.loc[cmaq_df.nunique(axis = 1) == 1].index)
    cmaq_df = cmaq_df.drop(bad_times)
    agg_df = agg_df.drop(bad_times)
    return cmaq_df, agg_df

### Remove CMAQ data where AEROMMA data is missing
The purpose of the `mk_cmaq_missing()` function is to replace valid CMAQ data with `np.nan` for times when the corresponding AEROMMA data also missing. This makes sure that all CMAQ data that gets used has a 1-to-1 correspondence with AEROMMA observations.

Parameters:  
* `cmaq_array` (vector) - any 1d array-like object containing CMAQ data
* `aeromma_array` (vector) - any 1d array-like object containing AEROMMA data

Returns:
* `cmaq_array` (np.ndarray) - a 1d numpy array with the same data as the original `cmaq_array` but with some data replaced with `np.nan` at indices where `aeromma_array` has the value of `np.nan`

In [ ]:
def mk_cmaq_missing(cmaq_array, aeromma_array):
    #force both arrays to be in np.ndarray format
    cmaq_array = np.array(cmaq_array)
    aeromma_array = np.array(aeromma_array)
    
    #reset values of the CMAQ array to NaN when the AEROMMA data is missing
    for i, aeromma in enumerate(aeromma_array):
        if np.isnan(aeromma) == True:
            cmaq_array[i] = np.nan
    
    return cmaq_array

### Calculate percentage of data that is missing
The purpose of `pct_missing()` is to calculate the percent of data in a DataFrame column that is missing.

Parameters:  
* `var_name` (str) - The name of the data column you would like to analyze
* `df` (pandas.DataFrame) - The DataFrame contining the data column you would like to analyze

returns:  
The percent of data in the selected data column (`df[var_name]`) that has a value of `np.nan`.

In [ ]:
def pct_missing(var_name, df):
    all_data = len(df[var_name])
    missing_data = len(df[var_name].loc[np.isnan(df[var_name])])
    return 100 * (missing_data / all_data)

### Stats Table
The purpose of `stats_table()` is to produce a table of statistics comparing a variable from CMAQ and an equivalent variable from AEROMMA. It also calculates the bias (CMAQ - AEROMMA) and the absolute error (|bias|) of all CMAQ values paired to AEROMMA observations and calculates the same statistics on the new bias and absolute error arrays. The statistics computed for each array are:  
* mean
* median
* standard deviation
* minimum
* maximum
* percent missing  

Parameters:  
* `cmaq_data` (vector like) - A vector of values representing data from CMAQ output
* `aeromma_data` (vector like) - A vector of values representing AEROMMA observations
* `var_name` (str) - Name of the variable being analyzed with the statistics
* `var_units` (str) - Units of the variable being analyzed with the statistics
* `save_txt` (str or bool) [Default: `False`] - If not `False`, the path of a new text file to save the resulting stats table to. No table will be printed if the value is changed from `False`.

Returns:
* stats_dict (dict) - a dictionary storing all the statistics shown in the table.

In [ ]:
# calc_stats is a function used for stats_table. Scroll down to view stats_table
def calc_stats(data, missing_data, stats_dict, key):
    if missing_data < 100:
        stats_dict[key] = {'mean': np.nanmean(data), 
                           'median': np.nanmedian(data), 
                           'std': np.nanstd(data), 
                           'q1': np.nanquantile(data, 0.25), 
                           'q3': np.nanquantile(data, 0.75), 
                           'min': np.nanmin(data), 
                           'max': np.nanmax(data)}
    else:
        stats_dict[key] = {'mean': np.nan, 
                           'median': np.nan, 
                           'std': np.nan, 
                           'q1': np.nan, 
                           'q3': np.nan, 
                           'min': np.nan, 
                           'max': np.nan}
    return stats_dict
        

def stats_table(cmaq_data, aeromma_data, var_name, var_units, save_txt = False):
    #create an empty dictionary to store the statistics
    stats_dict = {}

    #caclulate the percentage of missing data
    missing_data = pct_missing('var', pd.DataFrame({'var': cmaq_data}))
    
    #calculate CMAQ and AEROMMA statistics
    stats_dict = calc_stats(cmaq_data, missing_data, stats_dict, 'CMAQ')
    stats_dict = calc_stats(aeromma_data, missing_data, stats_dict, 'AEROMMA')

    #find the difference and absolute error between CMAQ and AEROMMA data
    bias_values = cmaq_data - aeromma_data
    abs_error = np.abs(bias_values)
    
    #calculate bias statistics
    stats_dict = calc_stats(bias_values, missing_data, stats_dict, 'bias')
    stats_dict = calc_stats(abs_error, missing_data, stats_dict, 'abs_error')

    #create statistics table
    table_title = 'Statistics for ' + var_name + ' [' + var_units + ']'
    if save_txt == False:
        print()
        print(table_title)
        print('-' * len(table_title))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Statistic     |      CMAQ       |     AEROMMA     |      Bias       | Absolute Error  |')
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Mean          |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['mean'], stats_dict['AEROMMA']['mean'], stats_dict['bias']['mean'], stats_dict['abs_error']['mean']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Median        |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['median'], stats_dict['AEROMMA']['median'], stats_dict['bias']['median'], stats_dict['abs_error']['median']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Std Dev       |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['std'], stats_dict['AEROMMA']['std'], stats_dict['bias']['std'], stats_dict['abs_error']['std']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Quartile 1    |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['q1'], stats_dict['AEROMMA']['q1'], stats_dict['bias']['q1'], stats_dict['abs_error']['q1']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Quartile 3    |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['q3'], stats_dict['AEROMMA']['q3'], stats_dict['bias']['q3'], stats_dict['abs_error']['q3']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Minimum       |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['min'], stats_dict['AEROMMA']['min'], stats_dict['bias']['min'], stats_dict['abs_error']['min']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Maximum       |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |' % (stats_dict['CMAQ']['max'], stats_dict['AEROMMA']['max'], stats_dict['bias']['max'], stats_dict['abs_error']['max']))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print('|  Missing Data  |  %12.4f%%  |  %12.4f%%  |  %12.4f%%  |  %12.4f%%  |' % (missing_data, missing_data, missing_data, missing_data))
        print('+----------------+-----------------+-----------------+-----------------+-----------------+')
        print()

    #save the table to a text file if the user requests it
    if save_txt != False:
        file = open(save_txt, 'w')
        file.write(table_title + '\n')
        file.write('-' * len(table_title) + '\n')
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Statistic     |      CMAQ       |     AEROMMA     |      Bias       | Absolute Error  |\n')
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Mean          |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['mean'], stats_dict['AEROMMA']['mean'], stats_dict['bias']['mean'], stats_dict['abs_error']['mean']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Median        |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['median'], stats_dict['AEROMMA']['median'], stats_dict['bias']['median'], stats_dict['abs_error']['median']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Std Dev       |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['std'], stats_dict['AEROMMA']['std'], stats_dict['bias']['std'], stats_dict['abs_error']['std']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Quartile 1    |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['q1'], stats_dict['AEROMMA']['q1'], stats_dict['bias']['q1'], stats_dict['abs_error']['q1']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Quartile 3    |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['q3'], stats_dict['AEROMMA']['q3'], stats_dict['bias']['q3'], stats_dict['abs_error']['q3']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Minimum       |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['min'], stats_dict['AEROMMA']['min'], stats_dict['bias']['min'], stats_dict['abs_error']['min']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Maximum       |  %13.5f  |  %13.5f  |  %13.5f  |  %13.5f  |\n' % (stats_dict['CMAQ']['max'], stats_dict['AEROMMA']['max'], stats_dict['bias']['max'], stats_dict['abs_error']['max']))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.write('|  Missing Data  |  %12.4f%%  |  %12.4f%%  |  %12.4f%%  |  %12.4f%%  |\n' % (missing_data, missing_data, missing_data, missing_data))
        file.write('+----------------+-----------------+-----------------+-----------------+-----------------+\n')
        file.close()
        print('Statistics table saved to: ' + save_txt)

    return stats_dict

# Plotting Functions

### Compare AEROMMA data and paired CMAQ output
The purpose of the `compare_plot()` function is to create a plot based on values from an AEROMMA flight and CMAQ values paired to the flight location in addition to the flight ID and the name of the variable the AEROMMA and CMAQ values represent. If the CMAQ data is accurately reproducing what AEROMMA observed, the plotted points should fall along the line y = x, which plots in black. Any point that does not fall along the line represents a disagreement between AEROMMA observations and CMAQ output.  
  
Parameters:  
* `var_name` (str) - the name of the variable being plotted from AEROMMA and CMAQ data
* `flight_values` (vector like) - a vector containing the values from an AEROMMA variable
* `cmaq_values` (vector like) - a vector of CMAQ values that are paired to an equivalent data AEROMMA vector
* `var_units` (str) - the units of the variable
* `title_start` (str) [Default: `''`] - Text included at the begining of the figure title. Title string is `title_start + 'CMAQ vs AEROMMA\n' + var_name + ' [' + var_units + ']'`
* `fig_side_length` (float) [Default: `3.5`] - Changes the size of the figure. The figure shape is set to always be a square, so this parameter changes the length of a side of the square.
* `save_fig` (str or bool) [Default: `False`] - The full path and filename to save the figure to. If no string is provided, the resulting figure will not be saved as a file.

Returns:
* `None`. Instead, a plot is created.

In [ ]:
def compare_plot(var_name, flight_values, cmaq_values, var_units, title_start = '', fig_side_length = 5, save_fig = False):  
    #remove missing values and force the CMAQ and AEROMMA data to be in np.ndarray format
    cmaq_values = mk_cmaq_missing(cmaq_values, flight_values)
    cmaq_values = [value for value in cmaq_values if np.isnan(value) == False]
    flight_values = [value for value in flight_values if np.isnan(value) == False]

    #if data exists...
    if len(cmaq_values) > 0 and len(flight_values) > 0:
        #find the maximum and minimum value of combined flight and CMAQ data
        combined_array = []
        for i in range(len(flight_values)):
            combined_array.append(flight_values[i])
            combined_array.append(cmaq_values[i])
        max_value = np.nanmax(combined_array)
        min_value = np.nanmin(combined_array)
        max_min_diff = max_value - min_value
        max_value_plus = np.int32(np.ceil(max_value + max_min_diff * 0.05))
        min_value_plus = np.int32(np.floor(min_value - max_min_diff * 0.05))
        max_to_min_array = [min_value_plus - 1 + n for n in range(max_value_plus - min_value_plus + 2)]

        #remove times with missing data
        points_df = pd.DataFrame({'x':flight_values, 'y':cmaq_values})
        points_df = points_df.loc[points_df.notna().all(axis = 1)]
        
        #calculate point density
        points_array = np.vstack([np.array(points_df['x']), np.array(points_df['y'])])
        try:
            density = sp_stat.gaussian_kde(points_array)(points_array)
            calc_density = True
        except:
            calc_density = False

    else:
        warn('Data provided does not contain any values to plot. Resulting plot will be empty.')
        min_value_plus = -1
        max_value_plus = 1
        max_to_min_array = [min_value_plus - 1 + n for n in range(max_value_plus - min_value_plus + 2)]


    #plot the flight and CMAQ values against each other
    fig = plot.Figure()
    plot.rcParams['figure.figsize'] = (fig_side_length + 1, fig_side_length)
    plot.title(title_start + 'CMAQ vs AEROMMA\n' + var_name + ' [' + var_units + ']')
    plot.xlabel('AEROMMA ' + var_name)
    plot.ylabel('CMAQ ' + var_name)
    if len(cmaq_values) > 0 and len(flight_values) > 0:
        if calc_density == True:
            points = plot.scatter(np.array(points_df['x']), np.array(points_df['y']), c = density, cmap = 'plasma', s = 10, zorder = 1)
            cbar = plot.colorbar(points, label = 'Point Density', ticks = [np.min(density), np.max(density)])
            cbar.ax.set_yticklabels(['Low', 'High'])
        elif calc_density == False:
            plot.scatter(np.array(points_df['x']), np.array(points_df['y']), color = 'g', s = 10, zorder = 1)
    plot.plot(max_to_min_array, max_to_min_array, color = 'k', zorder = 5)    
    plot.xlim((min_value_plus, max_value_plus))
    plot.ylim((min_value_plus, max_value_plus))
    for value in plot.gca().get_yticks():
        value = float(value)
        plot.plot([min_value_plus, max_value_plus], [value, value], color = 'k', linestyle = '--', linewidth = 0.5, zorder = 3)
        plot.plot([value, value], [min_value_plus, max_value_plus], color = 'k', linestyle = '--', linewidth = 0.5, zorder = 3)
    if save_fig != False:
        plot.savefig(save_fig)
    elif save_fig == False:
        plot.show()
    plot.close()

### Compare AEROMMA and CMAQ coordinate data
The purpose of the `coord_checks()` function is to produce a series of comparison plots (generated by `compare_plot()`) based on the coordinate variables within AEROMMA and CMAQ data. These data are automatically extracted when pulling out any paired CMAQ data and, therefore, will always be availabe when CMAQ data is extracted.  

Parameters:  
* `flight_id` (str) - the string identifying the AEROMMA flight currently being used.
* `cmaq_df` (pandas.DataFrame) - DataFrame containing exracted CMAQ data that has been paired to the AEROMMA flight with the listed flight ID
* `agg_df` (pandas.DataFrame) - DataFrame containing AEROMMA flight data with the listed flight ID aggregated to a one minute time step.
* `descriptions` (bool) [Default: `True`] - If set to `True`, the function will add additional information describing what the plot should look like if everything has paired correctly. If set to `False`, the function will not produce this additional information.
* `save_figs` (str or bool) [Default: `False`] - The absolute path of the directory to save the figure to. The file name is provided by the function. If no string is provided, the resulting figure will not be saved as a file.

Returns:
* `None`. Instead, a plot is created. 

In [ ]:
def coord_checks(flight_id, cmaq_df, agg_df, descriptions = True, save_figs = False):
    #CMAQ coordinate extraction
    cmaq_times = cmaq_df.index
    cmaq_lons = cmaq_df['LON']
    cmaq_lats = cmaq_df['LAT']
    cmaq_alts = cmaq_df['ALT']
    
    #AEROMMA coordinate extraction
    flight_times = agg_df.index
    flight_lons = agg_df['G_LONG']
    flight_lats = agg_df['G_LAT']
    flight_alts = agg_df['G_ALT']
    
    #converts date format to an integer
    cmaq_times = [int(datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S').strftime('%m%d%H%M%S')) for time in cmaq_times]
    flight_times = [int(datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S').strftime('%m%d%H%M%S')) for time in flight_times]
    
    #plot data
    print('-------------------------------------------------------------------------------------------------------------')
    print('Comparison of coordinate data in AEROMMA and CMAQ for ' + flight_id)
    if save_figs == False:
        compare_plot('Time', flight_times, cmaq_times, 'YYYYMMDDHHmmSS', title_start = flight_id + ' ')
    elif save_figs != False:
        compare_plot('Time', flight_times, cmaq_times, 'YYYYMMDDHHmmSS', title_start = flight_id + ' ', save_fig = os.path.join(save_figs, 'time_comparison.png'))
    if descriptions == True:
        print('In the case of the time plot, all plots should be exactly on the line y = x. The index of every row in') 
        print('the DataFrame is based on the time stamp of the AEROMMA observation so all values should match extactly')
        print('in each data set. Any variation from y = x represents a mismatch in the data and needs to be addressed.')
        print('To clarify, the label on the y-axis, "CMAQ Time", is misleading in this case because the time does not')
        print('come from CMAQ, it comes from AEROMMA.')
    # print()
    if save_figs == False:
        compare_plot('LON', flight_lons, cmaq_lons, '°', title_start = flight_id + ' ')
        compare_plot('LAT', flight_lats, cmaq_lats, '°', title_start = flight_id + ' ')
    elif save_figs != False:
        compare_plot('LON', flight_lons, cmaq_lons, '°', title_start = flight_id + ' ', save_fig = os.path.join(save_figs, 'latitude_comparison.png'))
        compare_plot('LAT', flight_lats, cmaq_lats, '°', title_start = flight_id + ' ', save_fig = os.path.join(save_figs, 'longitude_comparison.png'))
    if descriptions == True:
        print('For latitude and longitude, there should be some variation aroundthe line y = x, but only slight. Any')
        print('clear deviation from the line represents a mismatch in the data.')
    print()
    if save_figs == False:
        compare_plot('ALT', flight_alts, cmaq_alts, 'm', title_start = flight_id + ' ')
    elif save_figs != False:
        compare_plot('ALT', flight_alts, cmaq_alts, 'm', title_start = flight_id + ' ', save_fig = os.path.join(save_figs,  'altitude_comparison.png'))
    if descriptions == True:
        print('For altitude, expect some minor scatter around the line y = x, especially at higher altitudes. After a')
        print('certain altitude, the points should become spread out along lines of equal CMAQ height, representing')
        print('the grid point altitudes in the model. The values should all be in terms of height above sea level. If')
        print('a majority of values are clearly below the line y = x, there is a good chance the CMAQ extraction was')
        print('set to height above ground level. Otherwise, any other significant scatter is a sign of mismatched data.')
    print('-------------------------------------------------------------------------------------------------------------')

### Compare AEROMMA and CMAQ meteorological data
The purpose of the met_compare() function is to produce a series of comparison plots (generated by compare_plot()) based on the meteorological variables within AEROMMA and CMAQ data. These variables are:  
* Temperature (AEROMMA: `'T'`, CMAQ: `'TA'`)
* Pressure (AEROMMA: `'P'`, CMAQ: `'PRES'`)

If an output_vars list is passed to the CMAQ or AEROMMA extraction functions, these variables are not automatically extracted and will need to be added to the output_vars list for this function to run sucessfully.

Parameters:  
* `flight_id` (str) - the string identifying the AEROMMA flight currently being used.
* `cmaq_df` (pandas.DataFrame) - DataFrame containing exracted CMAQ data that has been paired to the AEROMMA flight with the listed flight ID
* `agg_df` (pandas.DataFrame) - DataFrame containing AEROMMA flight data with the listed flight ID aggregated to a one minute time step.
* `save_figs` (str or bool) [Default: `False`] - The absolute path of the directory to save the figure to. The file name is provided by the function. If no string is provided, the resulting figure will not be saved as a file.

Returns:
* `None`. Instead, a plot is created. 

In [ ]:
def met_compare(flight_id, cmaq_df, agg_df, save_figs = False):
    #CMAQ coordinate extraction
    cmaq_t = cmaq_df['TA']
    cmaq_p = cmaq_df['PRES'] / 100
    
    #AEROMMA coordinate extraction
    flight_t = agg_df['T'] / 100
    flight_p = agg_df['P'] / 100
    
    #plot data
    print('-------------------------------------------------------------------------------------------------------------')
    print('Comparison of coordinate data in AEROMMA and CMAQ for ' + flight_id)
    if save_figs == False:
        compare_plot('Temperature', flight_t, cmaq_t, 'K', title_start = flight_id + ' ')
        compare_plot('Pressure', flight_p, cmaq_p, 'Pa', title_start = flight_id + ' ')

    elif save_figs != False:
        compare_plot('Temperature', flight_t, cmaq_t, 'K', title_start = flight_id + ' ', save_fig = os.path.join(save_figs,'temperature_comparison.png'))
        compare_plot('Pressure', flight_p, cmaq_p, 'mb', title_start = flight_id + ' ', save_fig = os.path.join(save_figs, 'pressure_comparison.png'))
    print('-------------------------------------------------------------------------------------------------------------')

### Time Series
The purpose of the `time_series()` function is to create a plot of one or more variables with the same units over a period of time.

Parameters:
* `times` (vector like) - Provides a list of times all in the string format `'YYYY-MM-DD HH:mm:SS'`. This vector will provide the x-values for the plot.
* `values_lists` (list) - Provides a series of vector like objects, each vector within `values lists` representing a new time series. The values in these vectors should represent y-values for the plot. The number of time series plotted depends on the number of vectors added to `values_list`.
* `var_name_list` (list) - Each list item should be a string that represents the name of the variable in the corresponding values list in `values_lists`. These will be used at plot labels.
* `colors` (list or NoneType) [Default: `None`] - Each list item should be a matplotlib color name corresponding to the variable in `var_name_list`.
* `plot_title` (str or NoneType) [Default: `None`] - The title of the plot.
* `y_label` (str or NoneType) [Default: `None`] - The label on the y-axis.
* `time_range` (list or NoneType) [Default: `None`] - A list of two date strings that set the x-limits of the plot with the format `[start_date, end_date]`. The date strings should have the same date format as the date strings in the `times` vector.
* `save_fig` (str or bool) [Default: `False`] - The full path and filename to save the figure to. If no string is provided, the resulting figure will not be saved as a file.

Returns:
* `None`. Instead, a figure is created.

In [ ]:
def time_series(times, values_lists, var_name_list, colors = None, plot_title = None, y_label = None, time_range = None, save_fig = False):
    #convert times to a datetime format
    times = [datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in times]

    #convert_times to a datetime format if time_range is provided
    if time_range != None:
        time_range = [datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in time_range]
        
    #force values_lists vectors to be in list format
    for i, values_list in enumerate(values_lists):
        values_lists[i] = list(values_list)

    #find times between flights
    for i in range(len(times) - 1):
        if times[i + 1] - times[i] > datetime.timedelta(minutes = 5):
            times.insert(i + 1, times[i] + datetime.timedelta(minutes = 5))    #adds an extra time point to the list
            for j in range(len(values_lists)):
                values_lists[j].insert(i + 1, np.nan)    #adds a NaN value to the extra time point for each variable
    
    #Create the figure
    fig = plot.figure()

    if plot_title != None:
        plot.title(plot_title)
    if y_label != None:
        plot.ylabel(y_label)
    plot.xlabel('Time')
    if time_range != None:
        plot.xlim(time_range)
    
    #plot the data
    for i, values_list in enumerate(values_lists):
        if colors == None:
            plot.plot(times, values_list, label = var_name_list[i])
        else: 
            plot.plot(times, values_list, color = colors[i], label = var_name_list[i])
    plot.legend(bbox_to_anchor=(1.05, 1))
    plot.grid()
    
    #close the figure
    if save_fig != False:
        plot.savefig(save_fig)
    if save_fig == False:
        plot.show()
    plot.close()

### Vertical Comparison
The purpose of the `altitude_bins()` function is to add a column to a cleaned AEROMMA or CMAQ Pandas DataFrame that specifies what altitude bin a particular observation falls in. 

Parameters:  
* `clean_df` (pandas.DataFrame) - a fully cleaned Pandas DataFrame that contains data from an AEROMMA flight either from CMAQ or AEROMMA itself. In other functions, these DataFrames are often refered to as `cmaq_df`, for cleaned CMAQ data, or `agg_df` for cleaned AEROMMA data.
* `bin_width` (int) [Default: `500`] - The width of bins used to sort data based on altitude. 
* `first_bin_height` (int) [Default: `0`] - The height at which to start the bottom of the lowest level altitude bin.
* `pre_def_bins` (list or NoneType) [Default: `None`] - User defined list of int and/or float values that define the edges of each altitude bin. The list should define both the top and bottom of each bin. Therefore, the number of bins will be equal to `len(list) - 1`. However, by default, the behavior set by `pre_def_bins` is ignored and the bins are created by the `bin_width` and `first_bin_height` parameters.
* `add_to_df` (bool) [Default: `False`] - When set to `False`, the function will return a list of strings that represent the altitude bin of every observation in the dataset. When set to `True`, the function adds the list of strings to the DataFrame under the column name `'alt_bins'`.  

Returns:
* Depending on the value of `add_to_df`, `altitude_bins()` returns either a list of strings that indicate the altitude bin for every observation (when `add_to_df` is set to `False`) or, the same list is added to the `clean_df` DataFrame as a column named `'alt_bins'` and the whole DataFrame is returned.

Notes:
* It is imperative that for all DataFrames that will be compared against each other with the `plot_vert()` funtion, the settings for bin creation (the `bin_width`, `first_bin_height`, and `pre_def_bins`) be the same. Having differences will cause errors in the `plot_vert()` function.

In [ ]:
def altitude_bins(clean_df, bin_width = 500, first_bin_height = 0, pre_def_bins = None, add_to_df = False):
    #force all starting bin heights to be integers
    first_bin_height = int(first_bin_height)
    
    #extract altitude data depending on if the data is from CMAQ or AEROMMA
    if 'G_LONG' in clean_df.keys():    #AEROMMA data 
        alt_data = clean_df['G_ALT']
    else:                        #CMAQ data
        alt_data = clean_df['ALT']
    
    #create a list of bin values that represent the lowest value in the bin
    if pre_def_bins == None:
        max_height = alt_data.max()
        current_bin_height = first_bin_height
        bin_edge_values = []
        bin_names = []
        while current_bin_height - max_height < bin_width:
            bin_edge_values.append(current_bin_height)
            bin_names.append(str(current_bin_height) + ' < alt <= ' + str(current_bin_height + bin_width))
            current_bin_height += bin_width
        bin_edge_values.append(current_bin_height)
    else:
        bin_edge_values = pre_def_bins[:]
        bin_names = []
        for i in range(1, len(bin_edge_values)):
            bin_names.append(str(bin_edge_values[i - 1]) + ' < alt <= ' + str(bin_edge_values[i]))
    
    #sort DataFrame rows into bins
    bin_list = pd.cut(alt_data, bins = bin_edge_values, labels = bin_names)
    if add_to_df == False:
        return list(bin_list)
    elif add_to_df == True:
        clean_df['alt_bins'] = bin_list
        return clean_df

The purpose of the `plot_vert()` function is to produce a plot showing the variation of a given variable with altitude, potentially making comparisons between CMAQ and AEROMMA data or other kinds of comparison.

Parameters:
* `value_lists` (list) - A list of vector like objects where each vector in the list contains a series of values of the variable you would like to plot. Every vector within the `value_list` list repesents a new variable to be plotted.
* `bin_lists` (list) - A list of vector like objects where each vector in the list contains a series of altitude bin names. The altitude bin names correspond to the values in `value_lists`. The number of vectors in `value_lists` and `bin_lists` should be the same.
* `var_names` (list) - A list of strings that contain the variable name of each vector in `value_lists`. the length of `value_lists` and `var_names` should be the same.
* `colors` (list) - A list of strings where each represents the color you would like to plot a specific variable. The color strings correspond to the variable names in `var_names` as well as the vectors in `value_lists` and `bin_lists`. The strings can either be [matplotlib colors](https://matplotlib.org/stable/gallery/color/named_colors.html) or color [hex codes](https://htmlcolorcodes.com/). 
* `x_label` (str or NoneType) [Default: `None`] - The label on the x-axis of the vertical plot. If no string is provided, there will be no x-axis label on the plot.
* `title` (str or NoneType) [Default: `None`] - The title of the vertical plot. If no string is provided, there will be no plot title.
* `save_fig` (str or bool) [Default: `False`] - The full path and filename to save the figure to. If no string is provided, the resulting figure will not be saved as a file.

In [ ]:
def plot_vert(bin_list, value_lists, var_names, colors, x_label = None, title = None, save_fig = False):
    #force all lists of data and altitude vectors to be in list format
    bin_list = list(bin_list)
    for i in range(len(value_lists)):
        value_lists[i] = list(value_lists[i])
    
    #construct a list of unique altitude bins ordered from lowest to hightest altitude
    unique_bins = list(set(bin_list))    #creates a list of unique bin names
    low_bin_edge = sorted([int(bin_name.split(' ')[0]) for bin_name in unique_bins])    #creates a sorted list of integers storing the lowest value of each bin
    high_bin_edge = sorted([int(bin_name.split(' ')[-1]) for bin_name in unique_bins])    #creates a sorted list of integers storing the highest value of each bin
    ordered_unique_bins = []
    bin_centers = []
    bin_widths = []
    for low, high in zip(low_bin_edge, high_bin_edge):
        ordered_unique_bins.append(str(low) + ' < alt <= ' + str(high))
        bin_centers.append(np.mean([low, high]))
        bin_widths.append(high - low)
    bin_centers = np.array(bin_centers)
    
    #construct dictionaries to store data about each bin
    bin_data = {}    #dictionary containing all the values for each bin   
    bin_means = {}    #dictionary containing the mean value for each bin
    bin_medians = {}    #dictionary containing the median value for each bin
    for var_name in var_names:
        bin_means[var_name] = []
        bin_medians[var_name] = []
        for bin_name in ordered_unique_bins:
            bin_data[var_name + ', ' + bin_name] = []
    
    #sort data into bin_data and bin_means dictionaries
    for value_list, var_name in zip(value_lists, var_names):
        #sort each value into a list in bin_data so that all values for each bin for a variable
        #are included 
        for value, bin_name in zip(value_list, bin_list):
            bin_data[var_name + ', ' + bin_name].append(value)
        
        #Caluclate mean and median values within bin_data bins and assign the altitude bin the
        #mean value in bin_means and median value in bin_medians
        for bin_name in ordered_unique_bins:
            bin_values = list(bin_data[var_name + ', ' + bin_name])
            bin_values = [value for value in bin_values if np.isnan(value) == False]
            if len(bin_values) > 0:
                bin_means[var_name].append(np.mean(bin_values))
                bin_medians[var_name].append(np.quantile(bin_values, 0.5))
            else:
                bin_means[var_name].append(np.nan) 
                bin_medians[var_name].append(np.nan)
        
    #find the total number of observations in each bin
    bin_count = []
    for bin_name in ordered_unique_bins:
        non_nan_list = [value for value in bin_data[var_name + ', ' + bin_name] if np.isnan(value) == False]
        bin_count.append(len(non_nan_list))
                
    #calculate and store the 1st and 3rd quartile values for each bin
    bin_iqr = {}
    for var_name in var_names:
        for key in bin_data:
            if var_name in key:
                nan_check = [value for value in bin_data[key] if np.isnan(value) == False]
                if len(nan_check) > 0:
                    try:
                        bin_iqr[var_name + 'q1'].append(np.nanquantile(bin_data[key], 0.25))
                        bin_iqr[var_name + 'q3'].append(np.nanquantile(bin_data[key], 0.75))
                    except KeyError:
                        bin_iqr[var_name + 'q1'] = [np.nanquantile(bin_data[key], 0.25)]
                        bin_iqr[var_name + 'q3'] = [np.nanquantile(bin_data[key], 0.75)]
                else:
                    try:
                        bin_iqr[var_name + 'q1'].append(np.nan)
                        bin_iqr[var_name + 'q3'].append(np.nan)
                    except KeyError:
                        bin_iqr[var_name + 'q1'] = [np.nan]
                        bin_iqr[var_name + 'q3'] = [np.nan]
    
    #check if there is any data to plot
    available_data = False
    for value_list in value_lists:
        for item in value_list:
            if available_data == False:
                if np.isnan(item) == False:
                    available_data = True

    #find values to serve as plot axis max and mins if there is data to plot
    if available_data == True:
        #x-axis on the main plot
        all_quartiles = []
        for key in bin_iqr:
            all_quartiles += bin_iqr[key]
        for key in bin_means:
            if key != 'alt_bins':
                all_quartiles += bin_means[key]
        max_quartile = np.nanmax(all_quartiles)
        min_quartile = np.nanmin(all_quartiles)
        range_extension = (max_quartile - min_quartile) * 0.05
        max_x = max_quartile + range_extension
        min_x = min_quartile - range_extension
        
        #max value of the x-axis on the histogram plot
        hist_max = np.nanmax(bin_count) * 1.05
        
        #y-axis on the plot
        max_y = max(high_bin_edge)
        min_y = min(low_bin_edge)

    #if there is no data to plot, force the max and min values to the below values
    elif available_data == False:
        warn('Data provided does not contain any values to plot. Resulting plot will be empty.')
        max_x = 1
        min_x = 0
        max_y = 12000
        min_y = 0
        hist_max = 1

    
    #create figure
    fig = plot.figure()
    fig.set_size_inches(6.4, 4.8)
    
    gs = gridspec.GridSpec(ncols = 2, nrows = 1, width_ratios = [1, 4], wspace = 0.05)
    main_plot = fig.add_subplot(gs[1])
    hist_plot = fig.add_subplot(gs[0])

    #generate the main vertical plot
    if title != None:
        main_plot.set_title(title)
    if x_label != None: 
        main_plot.set_xlabel(x_label)
    main_plot.set_ylabel('Altitude [m ASL]')
    main_plot.set_xlim((min_x, max_x))
    main_plot.set_ylim((min_y, max_y))
    main_plot.yaxis.tick_right()
    main_plot.yaxis.set_label_position('right')
    for i in range(len(var_names)):
        main_plot.bar([-10], [-10], color = colors[i], label = var_names[i])    #This adds each variable to the legend with a color box instead of a color line. None of this appears in the final plot
        main_plot.plot(bin_medians[var_names[i]], bin_centers, color = colors[i], zorder = i + 1)
        main_plot.scatter(bin_medians[var_names[i]], bin_centers, color = colors[i], zorder = i + 1, s = 10)
        main_plot.scatter(bin_means[var_names[i]], bin_centers, color = colors[i], zorder = i + 1, s = 30, marker = '+')
        for n in range(len(bin_centers)):
            main_plot.plot([bin_iqr[var_names[i] + 'q1'][n], bin_iqr[var_names[i] + 'q3'][n]], [bin_centers[n], bin_centers[n]], color = colors[i], zorder = i + 1)
    main_plot.grid(zorder = 0)
    main_plot.scatter([], [], color = 'k', label = 'Median')    #This adds the symbol for mean to the legend
    main_plot.scatter([], [], color = 'k', marker = '+', label = 'Mean')    #this adds the symbol for median to the legend
    main_plot.plot([], [], color = 'k', label = 'IQR')    #this adds the symbol for median to the legend
    main_plot.legend(bbox_to_anchor = [1.53, 1.018])

    #generate the histogram of data points per bin
    hist_plot.set_xlabel('Values per\nAltitude Bin')
    hist_plot.set_ylabel('Altitude [m ASL]')
    hist_plot.set_ylim((min_y, max_y))
    hist_plot.set_xlim((0, hist_max))
    hist_plot.grid()
    hist_plot.barh(bin_centers, bin_count, height = bin_widths, color = 'cornflowerblue')

    if save_fig != False:
        plot.savefig(save_fig)
    elif save_fig == False:
        plot.show()
    plot.close()

### Spatial Comparison
The purpose of `spatial_plot()` is to show the spatial distribution of CMAQ bias relative to AEROMMA observations.

Parameters:  
* `cmaq_df` (pandas.DataFrame) - Pandas data frame containing at least CMAQ latitude and longitude data.
* `plot_values` (vector like) - An array of values to be plotted on a map.
* `plot_type` (str) [Default: `'diff'`] - the type of plot you would like to make. The options are `'diff'` (for when plotting the difference between two arrays) and `'raw'` (for when plotting a normal set of values not representing a difference betwwen two arrays).
* `plot_title` (str or NoneType) [Default: `None`] - If not `None`, the input string becomes the plot title
* `colorbar_name` (str) [Default: `'viridis'`] - If the `plot_type` is set to `'raw'`, the colorbar can be changed to any [option provided by Matplotlib](https://matplotlib.org/stable/users/explain/colors/colormaps.html).
* `fig_size` (tuple) [Default: `(10, 8)`] - Tuple that defines the figure size with the format (width, height)
* `save_fig` (str or bool) [Default: `False`] - The full path and filename to save the figure to. If no string is provided, the resulting figure will not be saved as a file.

Returns:
* `None`. Instead, a plot is produced.

Notes:
* The difference values are spatially binned by CMAQ grid column. If there is more than one AEROMMA observation that falls within a grid column, they will be averaged together to produce one mean difference value, regardless of the time or alitude the observation was taken.
* To prevent extreme values from skewing the colorbar on the resulting plot, the max and min values of the colorbar are set by the maximum magnitude of the 5 and 95 percentile values as opposed to the maximum magnitude value overall (The max and min color bar values always have the same magnitude so the center of the diverging colorbar is set as close to zero as possible). This means that there will likely be some points with a difference value beyond the range of the colorbar. However, these values should only represent 5-10% of the points.

In [ ]:
def spatial_plot(cmaq_df, plot_values, plot_type = 'diff', plot_title = None, colorbar_name = 'viridis', fig_size = (10, 8), save_fig = False):
    #force all plot values to be in np.ndarray format
    plot_values = np.array(plot_values)
    
    #check to see if there is any valid data in plot_values
    all_data_missing = True
    for item in plot_values:
        if np.isnan(item) == False:
            all_data_missing = False

    #warn the user if all values are missing
    if all_data_missing == True:
        warn('All AEROMMA data is missing. Resulting plot will be empty.')

    elif all_data_missing == False:
        #Create a DataFrame to hold the spatial data
        spatial_df = pd.DataFrame()
        spatial_df['coords'] = list(zip(cmaq_df['LON'], cmaq_df['LAT']))
        spatial_df['flight_values'] = plot_values

        #Create a list of unique coordinate pairs
        unique_coords = list(set(spatial_df['coords']))

        #find the average value for each coordinate pair
        coord_values = []
        for coord_pair in unique_coords:
            coord_values.append(np.mean(spatial_df['flight_values'].loc[spatial_df['coords'] == coord_pair]))

        #remove coord pairs and values where the value is NaN
        spatial_df = pd.DataFrame({'coords': unique_coords, 'flight_values': coord_values})
        spatial_df = spatial_df.loc[spatial_df['flight_values'].isnull() == False]

        #add lats and lons back to the DataFrame separately
        spatial_df['lons'] = [coord[0] for coord in spatial_df['coords']]
        spatial_df['lats'] = [coord[1] for coord in spatial_df['coords']]

        #find values to serve as plot axis max and mins
        #find max and min lat and lon values
        max_lat = spatial_df['lats'].max()
        min_lat = spatial_df['lats'].min()
        max_lon = spatial_df['lons'].max()
        min_lon = spatial_df['lons'].min()

        #find the range of lat and lon values
        lat_range = max_lat - min_lat
        lon_range = max_lon - min_lon

        #find the range of lat and lon values in meters
        lat_to_m = lat_range * 111320    #1 degree of latitude is approximately 111320 km
        lon_to_m = lon_range * 111320 * np.cos((np.pi / 180) * ((lat_range / 2) + min_lat))    #distance in longitude varys with latitude. The calulation from above is multiplied by the cosine of the middle latitude to compensate
        
        #find the larger of the two distances
        range_extender = 1.1    #this adds a buffer to the plot extent
        max_range = max(lat_to_m * range_extender, lon_to_m * range_extender)

        #find the max and min lat values to use for the plot extent
        lat_center = max_lat - (lat_range / 2)
        min_y_extent = lat_center - (max_range / 111320 / 2)    #this converts the max_range value back to degrees latitude and subracts half from the center latitude
        max_y_extent = lat_center + (max_range / 111320 / 2)    #this converts the max_range value back to degrees latitude and adds half to the center latitude

        #find the max and min lon values to use for the plot extent
        lon_center = max_lon - (lon_range / 2)
        min_x_extent = lon_center - (max_range / 111320 / np.cos((np.pi / 180) * ((lat_range / 2) + min_lat)) / 2)    #this converts the max_range value back to degrees longitude and subracts half from the center longitude
        max_x_extent = lon_center + (max_range / 111320 / np.cos((np.pi / 180) * ((lat_range / 2) + min_lat)) / 2)    #this converts the max_range value back to degrees longitude and adds half to the center longitude

        #find the max absolute value of the 5 and 95 precentile differences
        #to set the color bar limits. The max and min values are not used 
        #because their magnitudes are sometimes much larger than all other
        #values and would skew the color bar
        value_05 = np.nanquantile(np.array(spatial_df['flight_values']), 0.05)
        value_95 = np.nanquantile(np.array(spatial_df['flight_values']), 0.95)
        max_quantile_extreme = max(abs(value_05), abs(value_95))


    #create a figure to plot the data on
    fig = plot.figure(figsize = fig_size)
    fig.set_size_inches(10, 8)
    ax = plot.axes(projection = ccrs.PlateCarree())
    if plot_title != None:
        ax.set_title(plot_title)

    #add map features
    ax.add_feature(cfeature.LAND, facecolor = 'floralwhite', zorder = 1)
    ax.add_feature(cfeature.LAKES, facecolor = 'w', zorder = 2)
    ax.add_feature(cfeature.OCEAN, facecolor = 'w', zorder = 2)
    ax.add_feature(cfeature.STATES, edgecolor = 'k', zorder = 7)
    ax.add_feature(cfeature.COASTLINE, edgecolor = 'k', zorder = 7)

    #plot the data if there is any
    if all_data_missing == False:
        if plot_type == 'diff':
            value_plot = ax.scatter(spatial_df['lons'], spatial_df['lats'], c = spatial_df['flight_values'], cmap = 'coolwarm', s = 10, zorder = 10, vmin = -1 * max_quantile_extreme, vmax = max_quantile_extreme)
        if plot_type == 'raw':
            value_plot = ax.scatter(spatial_df['lons'], spatial_df['lats'], c = spatial_df['flight_values'], cmap = colorbar_name, s = 10, zorder = 10, vmin = value_05, vmax = value_95)

        ax.set_extent((min_x_extent, max_x_extent, min_y_extent, max_y_extent), crs = ccrs.PlateCarree())
        plot.colorbar(value_plot, orientation = 'horizontal', pad = 0.05, shrink = 0.7, extend = 'both')

    #is there is no data, constrain the map extent to the United States
    elif all_data_missing == True:
        ax.set_extent((-126, -66, 24, 50), crs = ccrs.PlateCarree())

    if save_fig != False:
        plot.savefig(save_fig)
    elif save_fig == False:
        plot.show()
    plot.close()   

### Plot value relative to HCN
The purpose of the `function_of_hcn()` function is to create a scatter plot that shows how a value changes with varying HCN. This is important when performing an analysis of wildfire smoke because it often has a much higher concentration of HCN than surrounding environments.

Parameters:  
* `hcn_values` (vector like) - a vector of hydrogen cyanide values corresponding to the values you would like to plot. Ideally, this vector should come from AEROMMA data since they are observations and will better represent when the real HCN values are high (likely cooresponding to wildfire smoke) as opposed to CMAQ which may not have an accurate forecast. CMAQ values can be used if desired though.
* `plot_values` (vector_like) - a vector of values you would like to plot against the concentration of HCN to see if there is any separation of values at higher and lower HCN concentrations.
* `hcn_units` (str) [Default: `'ppt'`] - The units of the HCN values plotted on the x-axis
* `y_label` (str or NoneType) [Default: `None`] - The label on the y-axis of the plot
* `title` (str or NoneType) [Default: `None`] - The title of the plot
* `save_fig` (str or bool) [Default: `False`] - The full path and filename to save the figure to. If no string is provided, the resulting figure will not be saved as a file.

Returns:
* `None`. Instead, a plot is produced.

In [ ]:
def function_of_hcn(hcn_values, plot_values, hcn_units = 'ppt', y_label = None, title = None, save_fig = False):
    #Check to see if there are values to plot
    no_hcn = True
    for value in hcn_values:
        if np.isnan(value) == False:
            no_hcn = False
    no_plot_values = True
    for value in plot_values:
        if np.isnan(value) == False:
            no_plot_values = False
    if no_hcn == True or no_plot_values == True:    
        no_data = True
        warn('Data provided does not contain any values to plot. Resulting plot will be empty.')
    else:
        no_data = False

    #set values for max and min y-value
    if no_data == False:
        y_range = np.nanmax(plot_values) - np.nanmin(plot_values)
        y_max = np.nanmax(plot_values) + y_range * 0.05
        y_min = np.nanmin(plot_values) - y_range * 0.05

        #remove times with missing data
        points_df = pd.DataFrame({'x':hcn_values, 'y':plot_values})
        points_df = points_df.loc[points_df.notna().all(axis = 1)]
        
        #calculate point density
        points_array = np.vstack([np.array(points_df['x']), np.array(points_df['y'])])
        density = sp_stat.gaussian_kde(points_array)(points_array)
        
    #plot figure
    fig = plot.figure()
    plot.title(title)
    plot.xlabel('HCN Concentration [' + hcn_units + ']')
    plot.ylabel(y_label)
    if no_data == False:
        plot.xlim((0, np.nanmax(hcn_values) * 1.05))
        plot.ylim((y_min, y_max))
        points = plot.scatter(points_df['x'], points_df['y'], c = density, cmap = 'plasma', s = 10, zorder = 1)
        cbar = fig.colorbar(points, label = 'Point Density', ticks = [np.min(density), np.max(density)])
        cbar.ax.set_yticklabels(['Low', 'High'])
        plot.axhline(y = 0, color = 'k', zorder = 2)
    if save_fig != False:
        plot.savefig(save_fig)
    elif save_fig == False:
        plot.show()
    plot.grid()
    plot.close()

# Package Testing

In [ ]:
# if __name__ == '__main__':
#     index_file_path = '/work/MOD3DEV/mpye/flight_pairing/AEROMMA_flight_CMAQ_indices.nc'
#     cmaq_data_dir_path = '/work/MOD3DEV/has/2023cracmm_ages/runs/20241216demodata/data/output_CCTM_v55_intel23.2_2023_12US4'
#     cmaq_output_type = 'METCRO3D'
#     if cmaq_output_type == 'CONC':
#         cmaq_output_vars = ['NO', 'HONO', 'O3']
#     elif cmaq_output_type == 'METCRO3D':
#         cmaq_output_vars = ['TA', 'PRES']
    
#     flight_output_vars = ['T', 'P']
#     flight_data_dir = '/work/MOD3DEV/mpye/flight_pairing/flight_data/'
    
#     for dir_name in sorted(os.listdir(flight_data_dir))[-18:]:
#         if 'AEROMMA' in dir_name:
#             mrg_file_path = flight_data_dir + dir_name + '/' + dir_name + '.csv'
#             sec_data = extract_flight_data(mrg_file_path, output_vars = flight_output_vars)
#             clean_sec_data = rm_1hz_negatives(sec_data)
#             flight_df = time_agg_data(clean_sec_data)

#             flight_id = flight_id_creator(mrg_file_path)
#             cmaq_df = cmaq_flight_pairing.extract_cmaq_flight_data(flight_id, index_file_path, cmaq_data_dir_path, output_vars = cmaq_output_vars, cmaq_output_type = cmaq_output_type)
#             cmaq_df, flight_df = rm_outside_model_times(cmaq_df, flight_df)

#             stats_table(cmaq_df['TA'], flight_df['T'] / 100, 'Temperature', 'K') 
#             #time_series(cmaq_df.index, [cmaq_df['TA'], flight_df['T'] / 100], ['CMAQ', 'AEROMMA'], colors = ['r', 'k'])                      
#             #spatial_df = spatial_diff_plot(cmaq_df, cmaq_df['TA'], flight_df['T'] / 100, flight_id + '\nCMAQ minus AEROMMA Temperature [K]')
